In [ ]:
import csv
import random

documents = []
with open('../../data/training/training-data.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        documents.append(row['chunk'])

documents = random.sample(documents, 1000)

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

# setup Chroma in-memory, for easy prototyping. Can add persistence easily!
client = chromadb.Client()

# Create collection. get_collection, get_or_create_collection, delete_collection also available!
model_name = "../../training/models/german-nq-granite-embedding-278m-multilingual/checkpoint-4560"
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=model_name)
collection = client.get_or_create_collection("documents",embedding_function=sentence_transformer_ef)

# Add docs to the collection. Can also update and delete. Row-based API coming soon!
collection.add(
    documents=documents, # we handle tokenization, embedding, and indexing automatically. You can skip that and add your own embeddings as well
    metadatas=[{"source": "array"}]*len(documents), # filter on these!
    ids=[str(i) for i in range(len(documents))], # unique for each doc
)


In [ ]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()

## LLM Configuration
base_url = 'url'
api_key = "key"
use_azure = True
model = 'gpt-4o-mini'

def init_client():
    if (use_azure):
        client = openai.AzureOpenAI(
                azure_endpoint=base_url,
                api_key=api_key,
                api_version="2024-08-01-preview"
        )
    else:
        client = openai.OpenAI(
                api_key=api_key,
                base_url=base_url
        )
    return client

def get_answer(prompt, gesetze, question):
    response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "system", "content": gesetze},
                    {"role": "user", "content": question}],
                temperature=0.7,  # Controls randomness
                max_tokens=1000    # Limits response length
            )
    return response.choices[0].message.content


client = init_client()
with open('answer-prompt.txt', 'r', encoding='utf-8') as f:
    prompt = f.read()

def find_gesetze(question):
        gesetze = collection.query(
            query_texts=[question], # query text
            n_results=5)['documents'][0]
        return "\\n".join(gesetze)

def answer(question):
    return get_answer(prompt, find_gesetze(question), question)

In [13]:
question = 'Wann ist der Bau meines Carports Verfahrensfrei?'
my_answer = answer(question)
print(my_answer)

Der Bau eines Carports ist in der Regel verfahrensfrei, wenn bestimmte Voraussetzungen erfüllt sind. Diese sind in der Sächsischen Bauordnung nicht explizit aufgeführt, aber es gibt allgemeine Regelungen für verfahrensfreie Vorhaben. 

In vielen Fällen sind Carports verfahrensfrei, wenn sie bestimmte Maße nicht überschreiten, wie z.B. eine maximale Fläche oder Höhe. Wenn der Carport nicht in einem bestimmten Abstand zu Grundstücksgrenzen errichtet wird, kann dies ebenfalls die Verfahrensfreiheit beeinflussen.

Um sicherzustellen, dass Ihr Carport tatsächlich verfahrensfrei ist, sollten Sie sich bei Ihrer örtlichen Bauaufsichtsbehörde erkundigen. Dort erhalten Sie genaue Informationen zu den spezifischen Anforderungen in Ihrer Gemeinde.

Bitte beachten Sie, dass ich Ihnen keine konkreten gesetzlichen Regelungen zu verfahrensfreien Carports bereitstellen kann, da diese Informationen in den bereitgestellten Gesetzestexten nicht vorhanden sind. Ich empfehle Ihnen, die örtlichen Vorschrifte